<a href="https://colab.research.google.com/github/kmAyush/Video-Dubbing-with-Lip-Synchronization/blob/main/Text%20Translation/BPE_BiDirectional_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gdown

In [68]:
import pandas as pd
import math
import pickle

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import  nltk.translate.bleu_score as bleu

from io import open
import unicodedata
import random
import tqdm.auto as tqdm

from nltk.translate import bleu_score
from torch.utils.data import TensorDataset, DataLoader

import json
import os
import gdown

In [11]:
url = "https://drive.google.com/file/d/1-7s8Kqr205fJQgl1m8Z36L3cykv0GKJG/view?usp=sharing"
id = url.split('/')[-2]
file_url = f'https://drive.google.com/uc?id={id}'
destination_location = "/content/Video-Dubbing-with-Lip-Synchronization/Text Translation/BPE/"
gdown.download(file_url , destination_location, quiet=False )

Downloading...
From: https://drive.google.com/uc?id=1-7s8Kqr205fJQgl1m8Z36L3cykv0GKJG
To: /content/Video-Dubbing-with-Lip-Synchronization/Text Translation/BPE/english.pkl
100%|██████████| 7.23M/7.23M [00:00<00:00, 168MB/s]


'/content/Video-Dubbing-with-Lip-Synchronization/Text Translation/BPE/english.pkl'

In [12]:
url = "https://drive.google.com/file/d/1-OjFAj8qMQnT_WNfq9mvkUguGuJogDVv/view?usp=sharing"
id = url.split('/')[-2]
file_url = f'https://drive.google.com/uc?id={id}'
destination_location = "/content/Video-Dubbing-with-Lip-Synchronization/Text Translation/BPE/"
gdown.download(file_url , destination_location, quiet=False )

Downloading...
From: https://drive.google.com/uc?id=1-OjFAj8qMQnT_WNfq9mvkUguGuJogDVv
To: /content/Video-Dubbing-with-Lip-Synchronization/Text Translation/BPE/hindi.pkl
100%|██████████| 15.0M/15.0M [00:00<00:00, 137MB/s]


'/content/Video-Dubbing-with-Lip-Synchronization/Text Translation/BPE/hindi.pkl'

In [46]:
url = "https://drive.google.com/file/d/1ZjhBUE0w-SB6kO9_GFrrRIqCzKzGtQHC/view?usp=sharing"

id = url.split('/')[-2]
file_url = f'https://drive.google.com/uc?id={id}'
destination_location = "/content/Video-Dubbing-with-Lip-Synchronization/Text Translation/"
gdown.download(file_url , destination_location, quiet=False )

Downloading...
From: https://drive.google.com/uc?id=1ZjhBUE0w-SB6kO9_GFrrRIqCzKzGtQHC
To: /content/Video-Dubbing-with-Lip-Synchronization/Text Translation/new_data.csv
100%|██████████| 7.62M/7.62M [00:00<00:00, 65.8MB/s]


'/content/Video-Dubbing-with-Lip-Synchronization/Text Translation/new_data.csv'

In [7]:
# Read the CSV file into a DataFrame
file_path = "/content/Video-Dubbing-with-Lip-Synchronization/Text Translation/new_data.csv"
df = pd.read_csv(file_path)
df = df.iloc[:,:2]
df = df.dropna()
df.head()

,English,Hindi
0,"(LAUGHS) Oh, you're getting a bill.","(हंसते हुए) ओह, आपको बिल मिल रहा है।"
1,"Welcome, Professor.","आपका स्वागत है, प्रोफेसर."
2,PROFESSOR X: Are we\ndestined down this path?,प्रोफेसर एक्स: क्या हम हैं?\nइस पथ पर नियति?
3,We are at site coordinates\napproximately 15H-...,हम साइट निर्देशांक पर हैं\nलगभग 15एच-32...
4,What was that?,वह क्या था?


In [55]:
class Tokenizer:
    """ Represents the tokenizer for text data.
        Provides methods to encode and decode strings (as instance or as a batch). """

    def __init__(self):
        """ Initializes a new tokenizer.

            Any variables required in intermediate operations are declared here.
            You will also need to define things like special tokens and other things here.

            All variables declared in this function will be serialized
                and deserialized when loading and saving the Tokenizer.
            """
        self.pad_t = '<pad>'
        self.unk = '<unk>'
        self.start = '<start>'
        self.end = '<end>'
        self.vocab = {}
        self.tokens = []
        self.ids = []
        self.merges = {}
        self.vocab_size = None
        self.num_merges = None
        self.end_token_number = None
        self.pad_token_number = None
    @classmethod
    def load(cls, path):
        """ Loads a pre-trained tokenizer from the given directory.
           This directory will have a tokenizer.pkl file that contains all the tokenizer variables.

        Args:
            path (str): Path to load the tokenizer from.
        """
        tokenizer_file = os.path.join(path, "tokenizer.pkl")

        if not os.path.exists(path) or not os.path.exists(os.path.join(path, "tokenizer.pkl")):
            raise ValueError(cls.load.__name__ + ": No tokenizer found at the specified directory")

        with open(tokenizer_file, "rb") as ifile:
            return pickle.load(ifile)

    def save(self, path):
        """ Saves a trained tokenizer to a given directory, inside a tokenizer.pkl file.

        Args:
            path (str): Directory to save the tokenizer in.
        """

        os.makedirs(path, exist_ok=True)
        with open(os.path.join(path, "tokenizer.pkl"), 'wb') as ofile:
            pickle.dump(self, ofile)

    def get_stats(self, ids):
      counts = {}
      for pair in zip(ids, ids[1:]):
        if pair[0] != self.end_token_number and pair[1] != self.end_token_number:
            counts[pair] = counts.get(pair, 0) + 1
      return counts

    def merge(self, ids, pair, idx):
      newids = []
      i = 0
      while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
          newids.append(idx)
          i += 2
        else:
          newids.append(ids[i])
          i += 1

      return newids

    def train(self, data, vocab_size):
        """ Trains a tokenizer to learn meaningful representations from input data.
            In the end, learns a vocabulary of a fixed size over the given data.
            Special tokens, if any, must not be counted towards this vocabulary.

        Args:
            data (list[str]): List of input strings from a text corpus.
            vocab_size (int): Final desired size of the vocab to be learnt.
        """
        self.vocab_size = vocab_size
        self.num_merges = vocab_size - 256 - 3
        self.start_token_number = vocab_size - 3
        self.end_token_number = vocab_size - 2
        self.pad_token_number = vocab_size - 1

        for name in data:
          temp_tokens = name.encode('utf-8')
          temp_tokens = list(map(int, temp_tokens))
          temp_tokens.append(self.end_token_number)
          self.tokens += temp_tokens

        self.ids = list(self.tokens)
        for i in range(self.num_merges):
          stats = self.get_stats(self.ids)
          pair = max(stats, key = stats.get)
          idx = 256 + i
          self.ids = self.merge(self.ids,pair, idx)
          self.merges[pair] = idx


    def pad(self, tokens, length):
        """ Pads a tokenized string to a specified length, for batch processing.

        Args:
            tokens (list[int]): Encoded token string to be padded.
            length (int): Length of tokens to pad to.

        Returns:
            list[int]: Token string padded to desired length.
        """
        special_tokens = self.get_special_tokens()
        pad_val = special_tokens[self.pad_t]
        while len(tokens) < length:
             tokens.append(pad_val)

        return tokens

    def unpad(self, tokens):
        """ Removes padding from a token string.

        Args:
            tokens (list[int]): Encoded token string with padding.

        Returns:
            list[int]: Token string with padding removed.
        """
        special_tokens = self.get_special_tokens()
        pad_val = special_tokens[self.pad_t]
        while tokens[-1] == pad_val:
           tokens.pop()

        return tokens

    def get_special_tokens(self):
        """ Returns the associated special tokens.

            Returns:
                dict[str, int]: Mapping describing the special tokens, if any.
                    This is a mapping between a string segment (token) and its associated id (token_id).
        """
        special_tokens = {self.end:self.end_token_number, self.pad_t : self.pad_token_number, self.start : self.start_token_number }
        return special_tokens


    def get_vocabulary(self):
        """ Returns the learnt vocabulary post the training process.

            Returns:
                dict[str, int]: Mapping describing the vocabulary and special tokens, if any.
                    This is a mapping between a string segment (token) and its associated id (token_id).
        """
        self.vocab = {idx: bytes([idx]) for idx in range(256)} # dict  of token id and
        special_tokens = self.get_special_tokens()
        vc = {v:k.encode('utf-8') for k,v in special_tokens.items()}
        self.vocab.update(vc)
        for (p0, p1), idx in self.merges.items():
            self.vocab[idx] = self.vocab[p0] + self.vocab[p1]

        temp_vocab = {val:key for key,val in self.vocab.items()}
        return temp_vocab


    def encode(self, string, add_start=True, add_end=True):
        """ Encodes a string into a list of tokens.

        Args:
            string (str): Input string to be tokenized.
            add_start (bool): If true, adds the start of sequence token.
            add_end (bool): If true, adds the end of sequence token.
        Returns:
            list[int]: List of tokens (unpadded).
        """
        tokens = []
        if add_start == True:
          tokens.append(self.start_token_number)

        tokens.extend(list(string.encode("utf-8")))

        if add_end  == True:
           tokens.append(self.end_token_number)
        while len(tokens) >= 2:
          stats = self.get_stats(tokens)
          pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
          if pair not in self.merges:
              break
          idx = self.merges[pair]
          tokens = self.merge(tokens, pair, idx)

        return tokens

    def decode(self, tokens, strip_special=True):
        """ Decodes a string from a list of tokens.
            Undoes the tokenization, returning back the input string.

        Args:
            tokens (list[int]): List of encoded tokens to be decoded. No padding is assumed.
            strip_special (bool): Whether to remove special tokens or not.

        Returns:
            str: Decoded string.
        """
        end_length = len(self.end)
        if strip_special:
            if tokens[0] == self.start_token_number:
                tokens = tokens[1:]
        t_text = b"".join(self.vocab[idx] for idx in tokens)
        text = t_text.decode("utf-8", errors="replace")

        if text.endswith(self.end):
          text = text[:-end_length]

        return text

In [49]:
def render_glyph(token):
    """ Renders a token, handling invalid bytes in a safe, error-proof manner. """

    token = token.decode('utf-8', errors='replace') if isinstance(token, bytes) else token
    return "".join([ c if unicodedata.category(c)[0] != "C" else f"\\u{ord(c):04x}" for c in token ])

def inverse_vocabulary(tokenizer):
    """ Generates an inverse vocabulary with rendered tokens.

    Args:
        tokenizer (Tokenizer): Tokenizer whose vocabulary must be used.
    """

    return { id: render_glyph(token) for token, id in tokenizer.get_vocabulary().items() }

def apply_inverse_vocab(tokens, inv_vocab):
    """ Decodes using the given inverse vocabulary.

    Args:
        tokens (list[int]): Tokens to process.
        inv_vocab (dict[int, str]): Inverse vocabulary for mapping ids to tokens.

    Returns:
        list[str]: Mapped token glyphs.
    """

    return [ inv_vocab[id] for id in tokens ]

In [57]:
file_path = '/content/Video-Dubbing-with-Lip-Synchronization/Text Translation/BPE/english.pkl'

with open(file_path, 'rb') as file:
    src_tokenizer_model = pickle.load(file)

file_path = '/content/Video-Dubbing-with-Lip-Synchronization/Text Translation/BPE/hindi.pkl'

with open(file_path, 'rb') as file:
    tgt_tokenizer_model = pickle.load(file)

eng_vocab = src_tokenizer_model.get_vocabulary()
hin_vocab = tgt_tokenizer_model.get_vocabulary()

In [52]:
pairs = [[ src_tokenizer_model.encode(row['English']) , tgt_tokenizer_model.encode(row['Hindi']) ] for index, row in df.iterrows()]
from sklearn.model_selection import train_test_split
pairs_train, pairs_rest = train_test_split(pairs, test_size=0.2, random_state=42)
pairs_valid, pairs_test = train_test_split(pairs_rest, test_size=0.5, random_state=42)

In [58]:
df_test = pd.DataFrame(pairs_test,columns = ['English','Hindi'])
df_train = pd.DataFrame(pairs_train,columns = ['English','Hindi'])

In [59]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        input_data, target_data = self.pairs[idx]

        return torch.tensor(input_data), torch.tensor(target_data)

def pad_collate(batch):
    input_seqs, target_seqs = zip(*batch)

    input_seqs_padded = torch.nn.utils.rnn.pad_sequence(input_seqs, batch_first=True)

    # Pad target sequences to the maximum length in the batch
    target_seqs_padded = torch.nn.utils.rnn.pad_sequence(target_seqs, batch_first=True)

    return input_seqs_padded, target_seqs_padded


dataset_tr = MyDataset(pairs_train)
dataset_val = MyDataset(pairs_valid)
dataset_te = MyDataset(pairs_test)

batch_size = 8
train_dataloader = DataLoader(dataset_tr, batch_size=batch_size, collate_fn=pad_collate)
test_dataloader = DataLoader(dataset_te, batch_size=batch_size, collate_fn=pad_collate)
valid_dataloader = DataLoader(dataset_val, batch_size=batch_size, collate_fn=pad_collate)

In [60]:
class AttentionModule(torch.nn.Module):
    """ Implements an attention module """

    # Feel free to add additional parameters to __init__
    def __init__(self, input_size):
        """ Initializes the attention module.
            Feel free to declare any parameters as required. """

        super(AttentionModule, self).__init__()

        # BEGIN CODE : attn.init

        # ADD YOUR CODE HERE
        self.W_key = nn.Linear((2*input_size), input_size)
        self.W_query = nn.Linear(input_size, input_size)
        self.W_value =nn.Linear((2*input_size), input_size)


        # END CODE

    def forward(self, encoder_outputs, decoder_hidden_state):
        """ Performs a forward pass over the module, computing attention scores for inputs.

        Args:
            encoder_outputs (torch.Tensor): Output representations from the encoder, of shape [batch_size?, src_seq_len, output_dim].
            decoder_hidden_state (torch.Tensor): Hidden state from the decoder at current time step, of appropriate shape as per RNN unit (with optional batch dim).

        Returns:
            torch.Tensor: Attentions scores for given inputs, of shape [batch_size?, 1, src_seq_len]
        """

        keys = self.W_key(encoder_outputs)  # batch,seq_length, hidden_size
        values = self.W_value(encoder_outputs)   # batch,seq_length, hidden_size

        # decoder_hidden_state ==  num_layer,N, hid_size

        num_layers = decoder_hidden_state.size(0)
        decoder_hidden_state = decoder_hidden_state[num_layers-1].unsqueeze(1)  # N , 1, hidden
        query = self.W_query(decoder_hidden_state)  #  batch,1 ,hidden_size
        query = query.transpose(1, 2)  # (batch size, hidden size, 1)

        weights = torch.matmul(keys, query)  # batch , sequence, 1
        weights = F.softmax(weights, dim = 1)   # (batch, sequence,  1)

        context_vector = weights * values  # N, seq_length , hidden

        context_vector = torch.sum(context_vector, dim = 1, keepdim = True) # N, 1, hidden

        return context_vector, weights

In [61]:
class RNNEncoderDecoderLMWithAttention(torch.nn.Module):
    """ Implements an Encoder-Decoder network, using RNN units, augmented with attention. """
    def __init__(self,src_vocab_size, tgt_vocab_size, embd_dims, hidden_size, num_layers=1, dropout=0.1, teacher_force_ratio = 1 ):
        """ Initializes the encoder-decoder network, implemented via RNNs.

        Args:
            src_vocab_size (int): Source vocabulary size.
            tgt_vocab_size (int): Target vocabulary size.
            embd_dims (int): Embedding dimensions.
            hidden_size (int): Size/Dimensions for the hidden states.
        """

        super(RNNEncoderDecoderLMWithAttention, self).__init__()

        # Dummy parameter to track the model device. Do not modify.
        self._dummy_param = torch.nn.Parameter(torch.Tensor(0), requires_grad=False)

        self.src_vocab_size = src_vocab_size
        self.tgt_vocab_size = tgt_vocab_size

        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.dropout_enc = nn.Dropout(dropout)
        self.dropout_dec = nn.Dropout(dropout)
        self.teacher_force_ratio = teacher_force_ratio
        self.enc_pad_index = src_vocab_size - 1

        self.tgt_pad_index = tgt_vocab_size -1
        self.tgt_start_index = tgt_vocab_size - 3
        self.tgt_end_index = tgt_vocab_size - 2

        self.attension_module = AttentionModule(hidden_size)

        self.encoder_embedding = torch.nn.Embedding(src_vocab_size, embd_dims)
        self.encoder_lstm = nn.LSTM(embd_dims, hidden_size,num_layers = num_layers , batch_first = True, dropout = dropout, bidirectional=True)

        self.decoder_embedding = torch.nn.Embedding(tgt_vocab_size, embd_dims)
        self.decoder_lstm = nn.LSTM(embd_dims+hidden_size, hidden_size,num_layers = num_layers, batch_first = True, dropout = dropout)
        self.decoder_output_fc = nn.Linear(hidden_size, embd_dims)
        self.dec_out_embed = nn.Linear(embd_dims,tgt_vocab_size, bias = False)

        self.dec_out_embed.weight = self.decoder_embedding.weight

    @property
    def device(self):
        return self._dummy_param.device

    def forward(self, inputs, decoder_inputs=None, decoder_hidden_state=None, output_attention=False ):
        """ Performs a forward pass over the encoder-decoder network.

            Accepts inputs for the encoder, inputs for the decoder, and hidden state for
                the decoder to continue generation after the given input.

        Args:
            inputs (torch.Tensor): tensor of shape [batch_size?, src_seq_len]
            decoder_inputs (torch.Tensor): Decoder inputs, as tensor of shape [batch_size?, 1]
            decoder_hidden_state (any): tensor to represent decoder hidden state from time step T-1.
            output_attention (bool): If true, this function should also return the
                associated attention weights for the time step, of shape [batch_size?, 1, src_seq_len].

        Returns:
            tuple[torch.Tensor, any]: output from the decoder, and associated hidden state for the next step.

            Decoder outputs should be log probabilities over the target vocabulary.

        Example:
        >>> model = RNNEncoderDecoderWithAttention(*args, **kwargs)
        >>> output, hidden = model(..., output_attention=False)
        >>> output, hidden, attn_weights = model(..., output_attention=True)
        """

        batch_size = inputs.size(0)
        seq_length = decoder_inputs.size(1) if decoder_inputs != None else inputs.size(1)
        encoder_input_embed = self.encoder_embedding(inputs)
        encoder_output, (encoder_hidden ,cell_encoder )= self.encoder_lstm(encoder_input_embed)

        decoder_hidden = encoder_hidden[:self.num_layers,:,:]
        decoder_cell = cell_encoder[:self.num_layers,:,:]
        decoder_input_test = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(self.tgt_start_index)


        pred_output_prob = []
        weights  = []
        for i in range(1,seq_length):

            decoder_input_embed = self.decoder_embedding(decoder_input_test)
            context_vector, wts = self.attension_module(encoder_output, decoder_hidden)  #in biderectional lstm out sieze(N,SEQ,2*HIDD)
            decoder_input_lstm = torch.cat((decoder_input_embed, context_vector), dim = 2)

            dec_output, (decoder_hidden,decoder_cell) = self.decoder_lstm(decoder_input_lstm, (decoder_hidden,decoder_cell))
            out_tgt = self.decoder_output_fc(dec_output)
            out_tgt_embd = self.dec_out_embed(out_tgt)
            pred_output = F.log_softmax(out_tgt_embd, dim = -1)

            if decoder_inputs != None:
              teacher_frc = torch.rand(1, 1)
              if teacher_frc < self.teacher_force_ratio:
                 decoder_input_test = decoder_inputs[:, i].unsqueeze(1)

              else:
                 _, topi = pred_output.topk(1)
                 decoder_input_test = topi.squeeze(-1).detach()
            else:
               _, topi = pred_output.topk(1)
               decoder_input_test = topi.squeeze(-1).detach()

            pred_output_prob.append(pred_output)
            wts = wts.squeeze(0).squeeze(1)
            weights.append(wts)


        pred_output_prob = torch.cat(pred_output_prob, dim = 1)
        weights = torch.stack(weights)


        return pred_output_prob,weights  #, decoder_hidden, decoder_cell , encoder_output

In [69]:
def sync_vram():
    """ Synchronizes the VRAM across the GPUs, reclaiming unused memory. """
    torch.cuda.empty_cache()
    torch.cuda.synchronize()

In [71]:
class Trainer:
    """ Performs model training in a model-agnostic manner.
        Requires specifying the model instance, the loss criterion to optimize,
          the optimizer to use and the directory to save data to.
    """

    def __init__(self, model, criterion, optimizer):
        """ Initializes the trainer.

        Args:
            directory (str): Directory to save checkpoints and the model data in.
            model (torch.nn.Module): Torch model (must inherit `torch.nn.Module`) to train.
            criterion (torch.nn.Function): Loss criterion, i.e., the loss function to optimize for training.
            optimizer (torch.optim.Optimizer): Optimizer to use for training.
        """

        self.model            = model
        self.optimizer        = optimizer
        self.criterion        = criterion
        self.last_checkpoint  = 0
        self.loss_history     = { 'train': [], 'valid': [] }
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def train_step(self, x_batch, y_batch):
        """ Performs a step of training, on the training batch.

        Args:
            x_batch (torch.Tensor): Input batch.
            y_batch (torch.Tensor): Output batch.

        Returns:
            float: Training loss with the current model, on this batch.
        """
        self.model.train()
        x_batch = x_batch.to(self.device)
        y_batch = y_batch.to(self.device)
        self.optimizer.zero_grad()
        score,_ = self.model(x_batch,y_batch)
        pred_size = score.size(2)
        y_batch = y_batch[:,1:]
        loss = self.criterion(score.view(-1,pred_size),y_batch.reshape(-1))
        loss.backward()
        self.optimizer.step()

        return loss.item()

    def eval_step(self, validation_dataloader):
        """ Perfoms an evaluation step, on the validation dataloader.

        Args:
            validation_dataloader (torch.utils.data.DataLoader): Dataloader for the validation dataset.

        Returns:
            float: Validation loss with the current model checkpoint.
        """
        self.model.eval()
        val_loss = 0
        for input, target in validation_dataloader:
           input = input.to(self.device)
           target = target.to(self.device)
           score,_ = self.model(input,target)
           pred_size = score.size(2)
           target = target[:,1:]
           loss = self.criterion(score.view(-1,pred_size),target.reshape(-1))
           val_loss += loss.item()

        evaluator1 = Evaluator()
        evaluator1.set_decoding_method(rnn_greedy_generate)
        evaluator1.evaluate( self.model,df_sample['English'], true_data)

        return  val_loss/len(validation_dataloader)

    def train(self, train_dataloader,  validation_dataloader =None,
              num_epochs=10, batch_size=8, shuffle=True,
              save_steps=100, eval_steps=100, collate_fn=None):
        """ Handles the training loop for the model.

        Args:
            train_dataset (torch.utils.data.Dataset): Dataset to train on.
            validation_dataset (torch.utils.data.Dataset, optional): Data to validate on. Defaults to None.
            num_epochs (int, optional): Number of epochs to train for. Defaults to 10.
            batch_size (int, optional): Number of items to process per batch. Defaults to 8.
            shuffle (bool, optional): Whether to shuffle the data or not. Defaults to True.
            save_steps (int, optional): Number of steps post which a checkpoint should be saved. Defaults to 100.
            eval_steps (int, optional): Number of steps post which the model should be evaluated. Defaults to 100.
            collate_fn (function, optional): Function to use for collating instances to a batch.
        """

        current_checkpoint = 0
        self.model.to(self.device)
        self.model.train()
        with tqdm.tqdm(total = math.ceil(len(train_dataset) / batch_size) * num_epochs) as pbar:
            for epoch in range(num_epochs):
                for batch, (x_batch, y_batch) in enumerate(train_dataloader):
                    pbar.set_description(f"Epoch {epoch+1} / {num_epochs}")
                    if current_checkpoint < self.last_checkpoint:
                        current_checkpoint += 1
                        pbar.update()
                        continue
                    loss = self.train_step(x_batch, y_batch)
                    self.loss_history['train'].append(loss)
                    pbar.set_postfix({ 'batch': batch+1, 'loss': loss })

                    current_checkpoint += 1
                    pbar.update()

                    # Evaluate after every eval_steps
                    if (current_checkpoint) % eval_steps == 0:
                        if validation_dataset is not None:
                            val_loss = self.eval_step(validation_dataloader)
                            self.loss_history['valid'].append(val_loss)
                        else:
                            val_loss = None

                        print('[>]', f"epoch #{epoch+1:{len(str(num_epochs))}},",
                              f"batch #{batch+1:{len(str(len(train_dataloader)))}}:",
                              "loss:", f"{loss:.8f}", '|', "val_loss:", f"{val_loss:.8f}")
                    if (current_checkpoint) % save_steps == 0:
                        self.save(current_checkpoint, { 'loss': loss, 'checkpoint': current_checkpoint })
                    sync_vram()

            self.save(current_checkpoint)


    def save(self, checkpoint=None, metadata=None):
        """ Saves an associated model or a training checkpoint.

            If a checkpoint is specified, saves a checkpoint specific directory with optimizer data
                so that training can be resumed post that checkpoint.

        Args:
            checkpoint (int, optional): Checkpoint index. Defaults to None.
            metadata (dict[str, any], optional): Additional metadata to save alongside a checkpoint. Defaults to None.
        """
        drive_path = '/content/Video-Dubbing-with-Lip-Synchronization/Text Translation/BPE/'
        if checkpoint is not None:
            checkpoint_dir = drive_path + 'checkpoint/'#os.path.join(self.directory, f"checkpoint-{checkpoint}")
            os.makedirs(checkpoint_dir, exist_ok=True)
            torch.save(self.model.state_dict(), os.path.join(checkpoint_dir, "model.pt"))
            torch.save(self.optimizer.state_dict(), os.path.join(checkpoint_dir, "optimizer.pt"))
            with open(os.path.join(checkpoint_dir, "loss.json"), "w+", encoding='utf-8') as ofile:
                json.dump(self.loss_history, ofile, ensure_ascii=False, indent=2)
            if metadata:
                with open(os.path.join(checkpoint_dir, "metadata.json"), "w+", encoding='utf-8') as ofile:
                    json.dump(metadata, ofile, ensure_ascii=False, indent=2)
        else:
            directory = '/content/Video-Dubbing-with-Lip-Synchronization/Text Translation/BPE/'
            torch.save(self.model, os.path.join(directory, "model.pt"))
            with open(os.path.join(directory, "loss.json"), "w+", encoding='utf-8') as ofile:
                json.dump(self.loss_history, ofile, ensure_ascii=False, indent=2)
            if metadata:
                with open(os.path.join(directory, "metadata.json"), "w+", encoding='utf-8') as ofile:
                    json.dump(metadata, ofile, ensure_ascii=False, indent=2)


In [72]:
def remove_paddings(sequence,pad_token):
  while(sequence[-1] == pad_token):
    sequence = sequence[:-1]

  return sequence

In [73]:
def rnn_greedy_generate(model, seq_x, max_length=5):

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    input_length = len(seq_x)

    for i in range(5):
      seq_x.append(0)
    model.eval()
    input_tokens = torch.tensor(seq_x).unsqueeze(0)
    input_tokens = input_tokens.to(device)
    output_probs,_ = model(input_tokens)
    tgt_tokens = torch.argmax(output_probs, dim = 2).squeeze(0).tolist()
    tgt_tokens = remove_paddings(tgt_tokens,0)
    trans_seq = tgt_tokenizer_model.decode(tgt_tokens[:-1])

    return trans_seq

In [74]:
class Evaluator:
    """ Class to handle all the logic concerning the evaluation of trained models.  """

    def __init__(self) -> None:
        """ Initializes the evaluator.

        Args:
            src_tokenizer (Tokenizer): Tokenizer for input strings in the source language.
            tgt_tokenizer (Tokenizer): Tokenizer for output strings in the target language.
        """

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.decoding_method = None

    def set_decoding_method(self, decoding_method):
        """ Sets the decoding method to use with models.
                The evaluation function will use the set decoding method to generate outputs from the model.

        Args:
            decoding_method (function): Decoding method.
                Must accept the model instance, the input string, and tokenizers as arguments.
                Can accept additional arguments if required.
        """

        self.decoding_method = decoding_method

    @staticmethod
    def decompose(string):
        """ Decomposes a string into a set of tokens.

        Args:
            string (str): String to decompose.

        Returns:
            list[str]: List of characters from the string.
        """
        return unicodedata.normalize('NFKD', string).encode('utf-8')

    @staticmethod
    def levenshtein_distance(string1, string2):
        """ Computes the levensthein distance between two strings.

        Args:
            string1 (list[any]): Sequence A.
            string2 (list[any]): Sequence B.

        Returns:
            tuple[int, int, int]: Number of insertions + deletions, substitutions and no-ops.
        """

        costs = [
            [ 0 for j in range(len(string2)+1) ]
            for i in range(len(string1)+1)
        ]

        # Prepare matrix of costs.
        for i in range(len(string1)+1): costs[i][0] = i
        for j in range(len(string2)+1): costs[0][j] = j
        for i in range(1, len(string1)+1):
            for j in range(1, len(string2)+1):
                costs[i][j] = min(
                    costs[i][j-1] + 1,
                    costs[i-1][j] + 1,
                    costs[i-1][j-1] + (0 if string1[i-1] == string2[j-1] else 1)
                )

        # Decode matrix in backward manner for actual operation counts.
        c_ins_del, c_sub, c_noop = 0, 0, 0

        i, j = len(string1), len(string2)
        while i > 0 or j > 0:
            if i > 0 and costs[i][j] == costs[i-1][j] + 1:
                c_ins_del += 1
                i -= 1
            elif j > 0 and costs[i][j] == costs[i][j-1] + 1:
                c_ins_del += 1
                j -= 1
            elif i > 0 and j > 0:
                if string1[i-1] == string2[j-1]:
                    c_noop += 1
                else:
                    c_sub += 1
                i, j = i-1, j-1
            else:
                break

        return c_ins_del, c_sub, c_noop

    @staticmethod
    def accuracy(y_true, y_pred):
        """ Computes the accuracy of the predictions, against a reference set of predictions.

        Args:
            y_true (list[str]): Actual translations.
            y_pred (list[str]): Generated translations.

        Returns:
            float: Accuracy score, between 0 and 1.
        """
        return sum(yi_true == yi_pred for yi_true, yi_pred in zip(y_true, y_pred)) / len(y_pred)

    @classmethod
    def char_error_rate(cls, y_true, y_pred):
        """ Computes the character level error rate (CER) of the set of
            predictions against the reference translations.

        Args:
            y_true (list[str]): Actual translations.
            y_pred (list[str]): Generated translations.

        Returns:
            float: CER score, between 0 and 1. Lower the better.
        """
        cer_score = 0
        for yi_true, yi_pred in zip(y_true, y_pred):
            yi_true, yi_pred = cls.decompose(yi_true), cls.decompose(yi_pred)
            c_ins_del, c_sub, c_noop = cls.levenshtein_distance(yi_true, yi_pred)
            cer_score += (c_ins_del + c_sub) / (c_ins_del + c_sub + c_noop)

        return cer_score / len(y_true)

    def token_error_rate(self, y_true, y_pred):
        """ Computes the token level error rate (TER) of the set of
            predictions against the reference translations.

        Args:
            y_true (list[str]): Actual translations.
            y_pred (list[str]): Generated translations.

        Returns:
            float: TER score, between 0 and 1. Lower the better.
        """

        ter_score = 0

        for yi_true, yi_pred in zip(y_true, y_pred):
            yi_true = tgt_tokenizer_model.encode(yi_true)
            yi_pred = tgt_tokenizer_model.encode(yi_pred)
            t_ins_del, t_sub, t_noop = self.levenshtein_distance(yi_true, yi_pred)
            ter_score += (t_ins_del + t_sub) / (t_ins_del + t_sub + t_noop)

        return ter_score / len(y_true)

    @classmethod
    def bleu_score(cls, y_true, y_pred):
        """ Computes the average BLEU score of the set of predictions against the reference translations.

            Uses default parameters and equal weights for all n-grams, with max N = 4. (Thus computes BLEU-4).
            Uses a smoothing method for the case of missing n-grams.

        Args:
            y_true (list[str]): Actual translations.
            y_pred (list[str]): Generated translations.

        Returns:
            float: BLEU-4 score, the higher the better.
        """

        y_true = [ [ cls.decompose(yi) ] for yi in y_true ]
        y_pred = [ cls.decompose(yi) for yi in y_pred ]

        smoothing = bleu_score.SmoothingFunction()

        return bleu_score.corpus_bleu(
            y_true, y_pred,
            smoothing_function=smoothing.method1
        )

    def evaluate(self, model, data, reference_outputs):
        """ Performs the evaluation of a specified model over given data.

        Args:
            model_path (str): Path to load the model from. Must have a model.pt file.
            data (list[str]): List of input strings to translate.
            reference_outputs (list[str]): List of output strings to use as reference.
            decoding_kwargs (dict[str, any]): Additional arguments to forward to the decoding method.
                This could be for instance, max_length for a greedy decoding method.

        Raises:
            ValueError: If the decoding method is not set apriori.
        """

        if self.decoding_method is None:
            raise ValueError(f"{self.evaluate.__name__}: no decoding method is set, assign before use.")
        model.to(self.device)
        model.eval()

        # Generate outputs.
        generated_outputs = []
        with torch.no_grad():
            for seq_x in data:
                generated_outputs.append(self.decoding_method(
                    model, seq_x))
        accuracy_score = self.accuracy(reference_outputs, generated_outputs)
        blue_score     = self.bleu_score(reference_outputs, generated_outputs)

        print("EVALUATION:", ">", "accuracy:", f"{accuracy_score:.2%}")
        print("EVALUATION:", ">", "BLEU    :", f"{blue_score:.4f}")
        print()

In [75]:
df_sample =df_test
true_data = [tgt_tokenizer_model.decode(tokens) for tokens in df_sample['Hindi']]
df_sample.shape

(6001, 2)

In [79]:
rnn_enc_dec_attn_params = {
    'src_vocab_size': 403,
    'tgt_vocab_size': 403,
    'embd_dims'     : 400,
    'hidden_size'   : 800,
    'dropout'       : 0.4,
    'num_layers'    : 4,
    'teacher_force_ratio' : 0.6
}

rnn_enc_dec_attn_training_params = dict(
    num_epochs=15,
    batch_size=8,
    shuffle=True,
    save_steps=500,
    eval_steps= 500
)
torch.manual_seed(42)
model = RNNEncoderDecoderLMWithAttention(**rnn_enc_dec_attn_params)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()
trainer = Trainer(
    model, criterion, optimizer
)

In [78]:
train_dataset      = train_dataloader
validation_dataset = valid_dataloader

rnn_enc_dec_attn_train_data = dict(
    train_dataset=train_dataset,
    validation_dataset=validation_dataset,
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
trainer.train(train_dataloader,valid_dataloader, **rnn_enc_dec_attn_training_params)

In [81]:
rnn_enc_dec_attn_params = {
    'src_vocab_size': 403,
    'tgt_vocab_size': 403,
    'embd_dims'     : 400,
    'hidden_size'   : 800,
    'dropout'       : 0.4,
    'num_layers'    : 4,
    'teacher_force_ratio' : 0.6
}
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_path = '/content/Video-Dubbing-with-Lip-Synchronization/Text Translation/BPE/model.pt'
model_state_dict = torch.load(model_path,, map_location = device)
model2 = RNNEncoderDecoderLMWithAttention(**rnn_enc_dec_attn_params)
model2.load_state_dict(model_state_dict)
model2 = model2.to(device)

In [ ]:
evaluator = Evaluator()
evaluator.set_decoding_method(rnn_greedy_generate)
evaluator.evaluate(
    model2,
    df_sample['English'], true_data
)

EVALUATION: > accuracy: 5.00%
EVALUATION: > BLEU    : 0.5962

